In [ ]:
import pandas as pd
import numpy as np
import h2o
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
h2o.init()

In [ ]:
from h2o.automl import H2OAutoML
from h2o.frame import H2OFrame

In [ ]:
df = pd.read_csv("Data Sheet 4.csv")
train_set = pd.read_csv("Data Sheet 2.csv")
test_set = pd.read_csv("Data Sheet 3.csv")

In [ ]:
print(train_set.shape)
print(test_set.shape)

In [ ]:
train_set = train_set.drop(columns = ['Unnamed: 0', 'country'])
test_set = test_set.drop(columns = ['Unnamed: 0', 'country'])

#### Polynomialize features

#### H2O

In [ ]:
train_h2o = H2OFrame(train_set)

In [ ]:
test_h2o = H2OFrame(test_set)

In [ ]:
x = train_h2o.columns
y = "gini"
x.remove(y)

In [ ]:
aml = H2OAutoML(max_models=20, max_runtime_secs=12000)
aml.train(x=x, y=y, training_frame=train_h2o)

In [ ]:
lb = aml.leaderboard
lb.head(rows=15)

In [ ]:
aml.leader #Best model

In [ ]:
model = h2o.get_model(lb[4,"model_id"])
varimp = model.varimp(use_pandas=True)

In [ ]:
varimp

In [ ]:
varimp.to_csv('FeatureImportanceTEST2.csv')

In [ ]:
model.varimp_plot()

In [ ]:
pred = aml.predict(test_h2o)
pred = pred[0].as_data_frame().values.flatten()

In [ ]:
# Assuming you've already trained your AutoML model and made predictions
# pred = aml.predict(test_h2o)

# Get the performance of the model on the test set
perf = aml.leader.model_performance(test_data=test_h2o)

# You can now print various performance metrics
r_squared = perf.r2()
mse = perf.mse()
rmse = perf.rmse()

print(f"R-squared: {r_squared}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")

In [ ]:
exa = aml.explain(test_h2o)